In [1]:
from torch import nn, save, load, argmax, no_grad
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms

In [2]:
train = datasets.CIFAR10(root='data',download=True,train=True,transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(32, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
]))
train_dataset = DataLoader(train,batch_size=128)

100%|██████████| 170498071/170498071 [00:06<00:00, 27169381.04it/s]


Extracting data/cifar-10-python.tar.gz to data


In [3]:

class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 512),
            nn.ReLU(),
            nn.Dropout(p=0.5),  # Regularization
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10)  # Output layer for CIFAR-10
        )

    def forward(self, x):
        return self.model(x)

In [4]:
device = 'cuda' # cuda for gpu

clf = ImageClassifier().to(device) #set to "cpu" if no gpu; clf ==> Classifier
opt = Adam(clf.parameters(), lr=1e-3, weight_decay = 1e-4)  # lr ==> learning rate; opt ==> Optimizer
loss_fn = nn.CrossEntropyLoss()

In [12]:
print(len(train_dataset))
epochs = 10
for epoch in range(epochs):
    clf.train()
    avg_loss = 0
    for batch_num,batch in enumerate(train_dataset):
        X,y = batch
        X, y = X.to(device), y.to(device)

        y_pred = clf(X)
        loss = loss_fn(y_pred,y)

        #Zero out the previous gradients
        opt.zero_grad()

        #Calculate backwards gradients
        loss.backward()

        #Step in the opp direction of gradient
        opt.step()

        if (batch_num+1) %100==0:
            print(f"Batch {batch_num+1} : {avg_loss/(batch_num)}")

        avg_loss += loss.item()

    print(f"Epoch {epoch+1} : {avg_loss/len(train_dataset)}")

with open('model_state.pt','wb') as f:
    save(clf.state_dict(),f)

391
Batch 100 : 0.596349154156868
Batch 200 : 0.611981759718315
Batch 300 : 0.6140652079646005
Epoch 1 : 0.6103677069744491
Batch 100 : 0.5867843937994254
Batch 200 : 0.6019211874235815
Batch 300 : 0.6049033462203864
Epoch 2 : 0.6024484190794513
Batch 100 : 0.5704756607913007
Batch 200 : 0.5825359295660527
Batch 300 : 0.5870951601494117
Epoch 3 : 0.5858540710280923
Batch 100 : 0.5780697814141861
Batch 200 : 0.5814148120245143
Batch 300 : 0.5845579729231704
Epoch 4 : 0.5841334994949038
Batch 100 : 0.5666660248029112
Batch 200 : 0.5766241337785769
Batch 300 : 0.5744518983523582
Epoch 5 : 0.5726010925934443
Batch 100 : 0.5450886593322561
Batch 200 : 0.5565792834339429
Batch 300 : 0.5623992396437604
Epoch 6 : 0.5580267184378241
Batch 100 : 0.5442735766521608
Batch 200 : 0.5471857910479733
Batch 300 : 0.5544660037576554
Epoch 7 : 0.5534398933810651
Batch 100 : 0.5345257839771232
Batch 200 : 0.5424205827054067
Batch 300 : 0.5458600720434285
Epoch 8 : 0.5443475574178769
Batch 100 : 0.54286778

In [13]:
test = datasets.CIFAR10(root='data', download=True, train = False,transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(32, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
]))
test_dataset = DataLoader(test,32)

total = 0
correct = 0

# Evaluate the model
clf.eval()  # Put the model in evaluation model
with no_grad():  # Disable gradient calculation for inference
    for batch_num, (X, y) in enumerate(test_dataset):
        X, y = X.to(device), y.to(device)

        # Get predictions
        y_pred = clf(X)

        # Take the index with the maximum logit value (predicted class)
        y_pred = argmax(y_pred, dim=1)

        # Update total number of samples
        total += y.size(0)

        # Count correct predictions
        correct += (y_pred == y).sum().item()

        # Print progress every 100 batches
        if (batch_num + 1) % 100 == 0:
            print(f"Processed {total} samples")

# Print final accuracy
print(f"Accuracy: {correct / total * 100:.2f}%")
print(f"Total correct predictions: {correct}")
print(f"Total samples: {total}")

Files already downloaded and verified
Processed 3200 samples
Processed 6400 samples
Processed 9600 samples
Accuracy: 80.00%
Total correct predictions: 8000
Total samples: 10000
